# Settings

In [1]:
!pip install -U datasets
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 73.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import random
from datasets import Dataset
import torch
from torch.optim import AdamW
import numpy as np
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

import pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Embedding 변환

In [5]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/hateslop_final/res/fragrantica_database.csv", sep=';', encoding='ISO-8859-1')

In [6]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder

embedding_model = SentenceTransformer("dawn78/minilm6_perfumerecommender_v4")
cross_model = CrossEncoder('dawn78/cross_encoder_v4', num_labels=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/24.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [32]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

class PerfumeVectorStore:
    def __init__(self, embedding_model_name, faiss_path, metadata_path):
        """FAISS 및 메타데이터 저장을 담당하는 클래스"""
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.faiss_path = faiss_path
        self.metadata_path = metadata_path

    def vectorize(self, data):
        """텍스트 데이터를 벡터화하여 반환"""
        return np.array(self.embedding_model.encode(data['notes'].tolist()))

    def save_embeddings(self, embeddings, data):
        """벡터 및 메타데이터 저장"""
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        faiss.write_index(index, self.faiss_path)

        metadata = data.to_dict(orient='records')
        with open(self.metadata_path, 'w') as f:
            json.dump({"metadata": metadata}, f)

        print(f"✅ {len(metadata)}개의 향수 데이터를 저장 완료!")

# 사용 예제
if __name__ == "__main__":
    import pandas as pd

    # 향수 데이터 로드 (예: CSV 파일 또는 직접 생성)
    data = pd.read_csv("/content/drive/MyDrive/hateslop_final/res/fragrantica_database.csv", sep=';', encoding='ISO-8859-1')

    # 벡터 저장 인스턴스 생성
    vector_store = PerfumeVectorStore(
        embedding_model_name="dawn78/minilm6_perfumerecommender_v3",
        faiss_path="/content/drive/MyDrive/hateslop_final/res/embeddings_index_tuned_notes.faiss",
        metadata_path="/content/drive/MyDrive/hateslop_final/res/metadata_tuned_notes.json"
    )

    # 벡터화 및 저장 실행
    embeddings = vector_store.vectorize(data)
    vector_store.save_embeddings(embeddings, data)


✅ 15892개의 향수 데이터를 저장 완료!


In [5]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder

class PerfumeRecommender:
    def __init__(self, embedding_model_name, cross_model_name, faiss_path, metadata_path):
        # 모델 로드
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.cross_model = CrossEncoder(cross_model_name, num_labels=1)

        self.faiss_path = faiss_path
        self.metadata_path = metadata_path

        # 🔹 FAISS 인덱스 및 메타데이터 로드
        self.index, self.metadata = self.load_embeddings()

    def load_embeddings(self):
        """FAISS 인덱스 및 메타데이터 파일 로드"""
        try:
            # 🔥 FAISS 인덱스 로드
            index = faiss.read_index(self.faiss_path)

            # 🔥 JSON 메타데이터 로드
            with open(self.metadata_path, "r") as f:
                metadata = json.load(f)["metadata"]

            print("✅ FAISS 인덱스 및 메타데이터 로드 완료!")
            return index, metadata

        except Exception as e:
            print("❌ 임베딩 데이터 로드 오류:", e)
            return None, None

    def rerank_results(self, user_query, results):
        """교차 인코더를 활용한 검색 결과 재정렬"""
        input_examples = [[user_query, result['description']] for result in results]
        relevance_scores = self.cross_model.predict(input_examples)
        ranked_results = sorted(zip(results, relevance_scores), key=lambda x: x[1], reverse=True)
        return [item[0] for item in ranked_results]

    def search_perfume(self, k=10, top_n=3):
        """메타데이터 필터링 후 FAISS 검색 및 재정렬"""
        if self.index is None or self.metadata is None:
            print("임베딩 데이터를 먼저 로드하세요.")
            return []

        # 🎯 1. 필터링 기준 정하기 (Brand, Country, Gender)
        def get_unique_values(column):
            """해당 컬럼의 유니크한 값 리스트 반환"""
            return sorted(set(item[column] for item in self.metadata if column in item))

        def ask_user_for_filter(column_name):
            """사용자에게 필터링할 값을 선택하게 함"""
            unique_values = get_unique_values(column_name)
            print(f"\n💡 {column_name} 선택 (Enter 입력 시 무시 가능)")
            print(f"  옵션: {', '.join(unique_values)}")

            user_input = input(f"{column_name}: ").strip()
            return user_input if user_input in unique_values else None

        # 🚀 필터링 기준 입력 받기
        selected_brand = ask_user_for_filter("Brand")
        selected_country = ask_user_for_filter("Country")
        selected_gender = ask_user_for_filter("Gender")
        query = input("Enter your query: ")

        # 🎯 2. 메타데이터 필터링 적용
        filtered_metadata = [
            item for item in self.metadata
            if (not selected_brand or item.get("Brand") == selected_brand)
            and (not selected_country or item.get("Country") == selected_country)
            and (not selected_gender or item.get("Gender") == selected_gender)
        ]

        if not filtered_metadata:
            print("❌ 필터링된 결과가 없습니다. 다시 시도하세요.")
            return []

        print(f"✅ {len(filtered_metadata)}개의 향수가 필터링되었습니다!")

        # 🎯 3. FAISS 검색 수행
        query_embedding = self.embedding_model.encode(query).reshape(1, -1)

        # 필터링된 데이터에 대한 FAISS 인덱스 생성 (동적 생성)
        filtered_embeddings = np.array([self.embedding_model.encode(item["description"]) for item in filtered_metadata])
        dimension = filtered_embeddings.shape[1]
        temp_index = faiss.IndexFlatL2(dimension)
        temp_index.add(filtered_embeddings)

        distances, indices = temp_index.search(query_embedding, k)

        # 🎯 4. 검색 결과 구성
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            result = filtered_metadata[idx]
            results.append({
                "description": result["description"],
                "distance": distance,
                "url": result["url"]
            })

        # 🎯 5. 교차 인코더로 검색 결과 재정렬
        reranked_results = self.rerank_results(query, results)[:top_n]

        # 🎯 6. 결과 출력
        for i, result in enumerate(reranked_results):
            print(f"\nResult {i + 1}:")
            print(f"Brand: {selected_brand if selected_brand else 'Any'}")
            print(f"Country: {selected_country if selected_country else 'Any'}")
            print(f"Gender: {selected_gender if selected_gender else 'Any'}")
            print(f"Description: {result['description']}")
            print(f"Cross-Encoder Score: {self.cross_model.predict([[query, result['description']]])[0]:.2f}")
            print(f"More information: {result['url']}")

        return reranked_results


In [7]:
# PerfumeRecommender 객체 생성
recommender_1 = PerfumeRecommender(
    embedding_model_name="dawn78/minilm6_perfumerecommender_v3",
    cross_model_name="dawn78/cross_encoder_v3",
    faiss_path="/content/drive/MyDrive/hateslop_final/res/embeddings_index_tuned_notes.faiss",
    metadata_path="/content/drive/MyDrive/hateslop_final/res/metadata_tuned_notes.json"
)

✅ FAISS 인덱스 및 메타데이터 로드 완료!


KeyboardInterrupt: Interrupted by user

In [8]:
# 향수 검색 실행
recommender_1.search_perfume()


💡 Brand 선택 (Enter 입력 시 무시 가능)
  옵션: a dozen roses, a lab on fire, a n other, aaron terence hughes, abaton, abdul karim al faransi, abel, abercrombie fitch, acca kappa, acqua di parma, acqua di portofino, adidas, adolfo dominguez, aedes de venustas, aesop, afnan, agatha ruiz de la prada, agatho parfum, agent provocateur, agonist, ajmal, akro, al haramain perfumes, al rehab, al wataniah, alessandro dell acqua, alexander mcqueen, alexandre j, alexandria fragrances, alfred dunhill, alfred sung, alghabra parfums, alyssa ashley, amouage, amouroud, amyi, anatole lebreton, and other stories, andrea maack, anfas, angel schlesser, angela ciampagna, angelos creations olfactives, animale, anka kus parfuem, anna sui, annayake, antonio alessandria, antonio banderas, aquolina, arabesque perfumes, arabian oud, aramis, ard al zaafaran, areej le dore, argos, ariana grande, armaf, armand basi, arquiste, arteolfatto, asdaaf, astrophil stella, atelier cologne, atelier des ors, atelier materi, atkinsons, a

[{'description': 'Incense Oud by By Kilian is a unisex fragrance featuring top notes of rose, cardamom, pink pepper, geranium, middle notes of patchouli, virginia cedar, papyrus, methyl pamplemousse, and base notes of incense, agarwood (oud), sandalwood, french labdanum, oak moss, musk. The main accords are warm spicy, woody, amber, smoky, rose. Released in 2011 from France, this fragrance has a rating of 4.37 out of 5 from 764 votes. Crafted by perfumer Sidonie Lancesseur. ',
  'distance': 0.25287256,
  'url': 'https://www.fragrantica.com/perfume/by-kilian/incense-oud-11090.html'},
 {'description': 'Musk Oud by By Kilian is a unisex fragrance featuring top notes of lemon, cardamom, coriander, middle notes of bulgarian rose, geranium, cypress, artemisia, and base notes of agarwood (oud), musk, rum, incense, patchouli. The main accords are aromatic, rose, oud, warm spicy, fresh spicy. Released in 2013 from France, this fragrance has a rating of 4.4 out of 5 from 833 votes. Crafted by pe

In [22]:
# PerfumeRecommender 객체 생성
recommender_2 = PerfumeRecommender(
    embedding_model_name="dawn78/minilm6_perfumerecommender_v4",
    cross_model_name="dawn78/cross_encoder_v4",
    faiss_path="/content/drive/MyDrive/hateslop_final/res/embeddings_index_tuned.faiss",
    metadata_path="/content/drive/MyDrive/hateslop_final/res/metadata_tuned.json"
)

# 향수 검색 실행
query = input("Enter your query: ")
recommender_2.search_perfume(query)

FAISS 인덱스 및 Metadata 로드 완료!
Enter your query: sexy fragrance with touch of vanila

💡 Brand 선택 (Enter 입력 시 무시 가능)
  옵션: a dozen roses, a lab on fire, a n other, aaron terence hughes, abaton, abdul karim al faransi, abel, abercrombie fitch, acca kappa, acqua di parma, acqua di portofino, adidas, adolfo dominguez, aedes de venustas, aesop, afnan, agatha ruiz de la prada, agatho parfum, agent provocateur, agonist, ajmal, akro, al haramain perfumes, al rehab, al wataniah, alessandro dell acqua, alexander mcqueen, alexandre j, alexandria fragrances, alfred dunhill, alfred sung, alghabra parfums, alyssa ashley, amouage, amouroud, amyi, anatole lebreton, and other stories, andrea maack, anfas, angel schlesser, angela ciampagna, angelos creations olfactives, animale, anka kus parfuem, anna sui, annayake, antonio alessandria, antonio banderas, aquolina, arabesque perfumes, arabian oud, aramis, ard al zaafaran, areej le dore, argos, ariana grande, armaf, armand basi, arquiste, arteolfatto, asdaaf

[{'description': 'Vanille Fatale 2024 by Tom Ford is a unisex fragrance featuring top notes of coriander, saffron, middle notes of coffee, barley, frangipani, narcissus, and base notes of madagascar vanilla, suede, tobacco, mahogany. The main accords are vanilla, tobacco, coffee, warm spicy, sweet. Released in 2024 from USA, this fragrance has a rating of 4.1 out of 5 from 248 votes.  ',
  'distance': 0.16787818,
  'url': 'https://www.fragrantica.com/perfume/tom-ford/vanille-fatale-2024-88659.html'},
 {'description': 'Vanille Fatale by Tom Ford is a unisex fragrance featuring top notes of rum, myrrh, olibanum, saffron, coriander, orange, lime, middle notes of barley, coffee, plum, frangipani, narcissus, artemisia, rose, and base notes of madagascar vanilla, suede, tobacco, mahogany, patchouli, oakmoss, violet. The main accords are vanilla, warm spicy, sweet, amber, woody. Released in 2017 from USA, this fragrance has a rating of 4.11 out of 5 from 2620 votes. Crafted by perfumer Yann V